### P5js graphing test
This notebook is to see how well p5js graphing (graphica.js) will work.

In [1]:
%%javascript
console.log("Let's roll!")

<IPython.core.display.Javascript object>

In [2]:
%%javascript

let myVar = 'test'
console.log(myVar)

<IPython.core.display.Javascript object>

In [3]:
%%javascript

element.text('Hello')

<IPython.core.display.Javascript object>

In [4]:
%%javascript
require.config({
    paths: {
        'p5': 'https://cdnjs.cloudflare.com/ajax/libs/p5.js/0.7.3/p5', 
        'p5.dom': 'https://cdnjs.cloudflare.com/ajax/libs/p5.js/0.7.3/addons/p5.dom',
        'lodash': 'https://cdnjs.cloudflare.com/ajax/libs/lodash.js/4.17.4/lodash.min'
    },
    shim: {
        'p5.dom': ['p5']
    }
});

<IPython.core.display.Javascript object>

In [5]:
%%javascript 

window.defineModule = function (name, dependencies, module) {
    // force the recreation of the module
    // (when re-executing a cell)
    require.undef(name);
    
    define(name, dependencies, module);
};

<IPython.core.display.Javascript object>

In [15]:
%%javascript

window.createSketchView = function (name, dependencies, module) {
    
    require.undef(name);
    
    define(name,
           ['@jupyter-widgets/base', 'p5', 'lodash'].concat(dependencies),
           (widgets, p5, _, ...deps) => {

        let viewName = `${name}View`;
        
        let View = widgets.DOMWidgetView.extend({
            initialize: function () {
                this.el.setAttribute('style', 'text-align: center;');
            },

            render: function () {
                // pass the model as the last dependency so it can
                // be accessed in the sketch
                let sketch = module(...deps, this.model);
                setTimeout(() => {
                    this.sketch = new p5(sketch, this.el);                    
                }, 0);
            },

            remove: function () {
                // stop the existing sketch when the view is removed
                // so p5.js can cancel the animation frame callback and free up resources
                if (this.sketch) {
                    this.sketch.remove();
                    this.sketch = null;
                }
            }
        });
        
        return {
            [viewName] : View,
        };
    });
}

<IPython.core.display.Javascript object>

In [16]:
%%javascript

// Test module defining a few constants, for example purposes
// Such constants should ideally be defined directly in the model
// and directly accessed by the view

defineModule('testModule', [], () => {
    const [W, H] = [500, 500];
    return {W, H};
})

<IPython.core.display.Javascript object>

In [17]:
%%javascript

createSketchView('Sketch2D', ['testModule', 'p5.dom'], (TestModule, module) => {
    return function(p) {
        const {W, H} = TestModule;
        const [CX, CY] = [W / 2, H / 2];
        let rs;
        console.log("p:", p)
        console.log("module:", module)
        
        p.setup = function(){
            p.createCanvas(W, H);
            p.rectMode(p.CENTER);
            rs = p.createSlider(0, 20, 2);
            rs.position(0,0);
        }

        p.draw = function () {
            p.background('#ddd');
            p.translate(CX, CY);
            //let n = model.get('n_squares');
            let n = rs.value();
            _.range(n).forEach(i => {
                p.push();
                p.rotate(p.frameCount / 200 * (i + 1));
                p.fill(i * 5, i * 100, i * 150);
                p.rect(0, 0, 200, 200);
                p.pop();
            });
        }
    };
})

<IPython.core.display.Javascript object>

In [18]:
import ipywidgets as widgets
from traitlets import Unicode, Int


class Sketch2D(widgets.DOMWidget):
    _view_name = Unicode('Sketch2DView').tag(sync=True)
    _view_module = Unicode('Sketch2D').tag(sync=True)
    _view_module_version = Unicode('0.1.0').tag(sync=True)
    n_squares = Int(1).tag(sync=True)

In [19]:
sketch_2d = Sketch2D()
sketch_2d

Sketch2D()